In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold
import math
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest
#ARI: Arizona Cardinals
#ATL: Atlanta Falcons
#BAL: Baltimore Ravens
#BUF: Buffalo Bills
#CAR: Carolina Panthers
#CHI: Chicago Bears
#CIN: Cincinnati Bengals
#CLE: Cleveland Browns
#DAL: Dallas Cowboys
#DEN: Denver Broncos
#DET: Detroit Lions
#GB: Green Bay Packers
#HOU: Houston Texans
#IND: Indianapolis Colts
#JAX: Jacksonville Jaguars
#KC: Kansas City Chiefs
#MIA: Miami Dolphins
#MIN: Minnesota Vikings
#NE: New England Patriots
#NO: New Orleans Saints
#NYG: New York Giants
#NYJ: New York Jets
#OAK: Oakland Raiders
#PHI: Philadelphia Eagles
#PIT: Pittsburgh Steelers
#SD: San Diego Chargers
#SEA: Seattle Seahawks
#SF: San Francisco 49ers
#STL: Saint Louis Rams
#TB: Tampa Bay Buccaneers
#TEN Tennessee Titans
#WAS: Washington Redskins

In [3]:
train_df = pd.read_csv(r'C:\Users\38980\OneDrive\Desktop\study\kaggle\NFL\train.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# wind 
def give_me_WindSpeed(x):
        x = str(x)
        x = x.replace('mph', '').strip()
        if '-' in x:
            x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
        elif 'gusts up to' in x:
            x = (int(x.split()[0]) + int(x.split()[-1])) / 2
        elif 'clam' in x:
            x = 0
        try:
            return float(x)
        except:
            return -99
train_df['Cleaned_WindSpeed'] = train_df['WindSpeed'].apply(give_me_WindSpeed)

In [37]:
#correct time
train_df['Cleaned_TimeHandoff'] = train_df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train_df['Cleaned_TimeSnap'] = train_df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train_df['Cleaned_PlayerBirthDate'] = train_df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
train_df['Cleaned_GameClock'] = train_df['GameClock'].apply(lambda x: datetime.datetime.strptime(x[:5], "%M:%S"))

In [6]:
#player age
seconds_in_year = 60*60*24*365.25
train_df['PlayerAge'] = train_df.apply(lambda row: (row['Cleaned_TimeHandoff']-row['Cleaned_PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)

In [7]:
#player height
train_df['Cleaned_PlayerHeight'] = train_df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

In [101]:
#team abbreviations correct
train_df.loc[train_df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
train_df.loc[train_df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
train_df.loc[train_df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
train_df.loc[train_df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
train_df.loc[train_df['FieldPosition'] == 'ARZ', 'PossessionTeam'] = 'ARI'
train_df.loc[train_df['FieldPosition'] == 'BLT', 'PossessionTeam'] = 'BAL'
train_df.loc[train_df['FieldPosition'] == 'CLV', 'PossessionTeam'] = 'CLE'
train_df.loc[train_df['FieldPosition'] == 'HST', 'PossessionTeam'] = 'HOU'

In [8]:
#Turf
Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
train_df['Cleaned_Turf'] = train_df['Turf'].map(Turf)

In [9]:
#fix weather
train_df['Cleaned_GameWeather'] = train_df['GameWeather'].str.lower()
indoor = "indoor"
train_df['Cleaned_GameWeather'] = train_df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
train_df['Cleaned_GameWeather'] = train_df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
train_df['Cleaned_GameWeather'] = train_df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
train_df['Cleaned_GameWeather'] = train_df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)

In [10]:
train_df['ToLeft'] = train_df.PlayDirection == "left"
train_df['IsBallCarrier'] = train_df.NflId == train_df.NflIdRusher

In [11]:
def get_dx_dy(angle, dist):
    radian_angle = (90 - angle)* math.pi/180.0
    dx = dist * math.cos(radian_angle)
    dy = dist * math.sin(radian_angle)
    return dx, dy

In [14]:
train_df['TeamOnOffense'] = "home"
train_df.loc[train_df.PossessionTeam != train_df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
train_df['IsOnOffense'] = train_df.Team == train_df.TeamOnOffense # Is player on offense?
train_df['YardLine_std'] = 100 - train_df.YardLine
train_df.loc[train_df.FieldPosition.fillna('') == train_df.PossessionTeam,  
          'YardLine_std'
         ] = train_df.loc[train_df.FieldPosition.fillna('') == train_df.PossessionTeam,  
          'YardLine']
train_df['X_std'] = train_df.X
train_df.loc[train_df.ToLeft, 'X_std'] = 120 - train_df.loc[train_df.ToLeft, 'X'] 
train_df['Y_std'] = train_df.Y
train_df.loc[train_df.ToLeft, 'Y_std'] = 160/3 - train_df.loc[train_df.ToLeft, 'Y'] 
train_df['Orientation_std'] = train_df.Orientation
train_df.loc[train_df.ToLeft, 'Orientation_std'] = np.mod(180 + train_df.loc[train_df.ToLeft, 'Orientation_std'], 360)
train_df.loc[train_df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + train_df.loc[train_df['Season'] == 2017, 'Orientation_std'], 360)
train_df['Dir_std'] = train_df.Dir
train_df.loc[train_df.ToLeft, 'Dir_std'] = np.mod(180 + train_df.loc[train_df.ToLeft, 'Dir_std'], 360)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in remainder
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in remainder
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in remainder


In [15]:
Rusher = train_df.loc[train_df['IsBallCarrier'],['S','A','Dis','PlayerAge','Cleaned_PlayerHeight'\
                   ,'PlayerWeight','X_std','Y_std','Dir_std','YardLine_std']]

In [40]:
external_factor = train_df.loc[train_df['IsBallCarrier'],\
                               ['Team','YardLine_std','Quarter','Cleaned_GameClock'\
                               'Down','Distance','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                               'OffenseFormation','OffensePersonnel','DefendersInTheBox',\
                               'DefensePersonnel','TimeHandoff','TimeSnap']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [39]:
train_df[['Quarter','GameClock','Cleaned_GameClock']].head(10200)

,Quarter,GameClock,Cleaned_GameClock
0,1,14:14:00,1900-01-01 00:14:14
1,1,14:14:00,1900-01-01 00:14:14
2,1,14:14:00,1900-01-01 00:14:14
3,1,14:14:00,1900-01-01 00:14:14
4,1,14:14:00,1900-01-01 00:14:14
5,1,14:14:00,1900-01-01 00:14:14
6,1,14:14:00,1900-01-01 00:14:14
7,1,14:14:00,1900-01-01 00:14:14
8,1,14:14:00,1900-01-01 00:14:14
9,1,14:14:00,1900-01-01 00:14:14
